In [95]:
import pandas as pd

## Base tuning

In [244]:
with open('logs_video1.txt') as f:
    lines1 = f.readlines()

In [245]:
with open('logs_video2.txt') as f:
    lines2 = f.readlines()

In [246]:
with open('logs_video3.txt') as f:
    lines3 = f.readlines()

In [247]:
lines = lines1 + lines2 + lines3

In [248]:
alpha = 5
models = []
keys = []
alphas = []
topks = []
mcs = []
hrs = []
for line in lines:
    if 'Model' in line:
        model = line.split(':')[-1].strip()
    if 'Hidden' in line:
        for i in range(alpha-1):
            for n in [5,10,20]:
                keys.append(line.split(':')[-1].strip())
                models.append(model)
                alphas.append(i+2)
                topks.append(n)
    if 'HR' in line:
        mcs.append(float(line.split(':')[-1].strip()))
        hrs.append(float(line.split(':')[1].strip().split(', ')[0]))

In [249]:
batches = np.array([[2**i] * (len(keys) // 6) for i in range(4,10)]).flatten()

In [252]:
res = pd.DataFrame([models, batches, keys, alphas, topks, mcs, hrs]).T
res.columns = ['model', 'batch_size', 'hidden_size', 'alpha', 'top_k', 'mcc', 'hr']

In [253]:
res.head()

,model,batch_size,hidden_size,alpha,top_k,mcc,hr
0,triangular matrix,16,"(16, 3)",2,5,0.008778,0.001752
1,triangular matrix,16,"(16, 3)",2,10,0.016288,0.006008
2,triangular matrix,16,"(16, 3)",2,20,0.020803,0.009762
3,triangular matrix,16,"(16, 3)",3,5,0.01286,0.001752
4,triangular matrix,16,"(16, 3)",3,10,0.023863,0.006008


In [254]:
best = res.groupby('alpha')['mcc'].max()

In [255]:
best

alpha
2    0.046070
3    0.058639
4    0.050983
5    0.046907
Name: mcc, dtype: float64

In [258]:
res[(res['alpha']==2)&(res['top_k']==10)].sort_values('hr', ascending=False)

,model,batch_size,hidden_size,alpha,top_k,mcc,hr
973,triangular matrix,32,"(1024, 3)",2,10,0.034085,0.025782
1549,square root matrix,32,"(32, 5)",2,10,0.030141,0.020275
469,triangular matrix,16,"(1024, 3)",2,10,0.029951,0.020025
1765,square root matrix,32,"(16, 5)",2,10,0.028049,0.01577
2473,square root matrix,64,"(512, 5)",2,10,0.007389,0.012265
...,...,...,...,...,...,...,...
1273,triangular matrix,32,"(16, 4)",2,10,NaN,0.0
5569,encoder and decoder layers different,512,"(32, 4)",2,10,None,None
5581,encoder and decoder layers different,512,"(32, 5)",2,10,None,None
5593,encoder and decoder layers different,512,"(64, 3)",2,10,None,None


In [206]:
res[res['mcc']==best.loc[2]]

,batch_size,hidden_size,alpha,top_k,mcc
1549,512,"(512, 3)",2,10,0.024323


In [207]:
res[res['mcc']==best.loc[3]]

,batch_size,hidden_size,alpha,top_k,mcc
137,16,"(128, 5)",3,20,0.024615


In [208]:
res[res['mcc']==best.loc[4]]

,batch_size,hidden_size,alpha,top_k,mcc
739,64,"(1024, 4)",4,10,0.03477


In [209]:
res[res['mcc']==best.loc[5]]

,batch_size,hidden_size,alpha,top_k,mcc
442,32,"(512, 3)",5,10,0.035572


In [210]:
res[res['mcc']==best.loc[6]]

KeyError: 6

## AE tuning

## AE results

In [194]:
with open('food_easer_res.txt') as f:
    lines = f.readlines()

In [195]:
lines

['failed to get console mode for stdout: The handle is invalid.\n',
 'Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple\n',
 'Collecting torch==1.13.1\n',
 '  Downloading https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/packages/packages/torch/1.13.1/torch-1.13.1-cp38-cp38-manylinux1_x86_64.whl (887.4 MB)\n',
 '     в”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓ 887.4/887.4 MB 289.3 MB/s eta 0:00:00\n',
 'Collecting scipy==1.10.1\n',
 '  Downloading https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/packages/packages/scipy/1.10.1/scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)\n',
 '     в”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓв”Ѓ 34.5/34.5 MB 290.6 MB/s eta 0:00:00\n',
 'Collecting numpy==1.22.2\n',
 '  Downloading https://artifactory.tcsbank.ru/artifa

In [116]:
lines2 = ['HR@5:0.026467203682393557, MRR@5: 0.012514384349827386, Coverage@5: 0.15633264766382604, NCDG@5: 0.015919932393721466, HR_pos@5: 0.025891829689298044, HR_neg@5: 0.0005753739930955121, MRR_pos@5: 0.01232259301879555, MRR_neg@5: 0.00019179133103183735, NCDG_pos@5: 0.01563224539717371, NDCG_neg@5: 0.00028768699654775604, Matthews@5: 0.026765332823572463\n',
 'HR@10:0.061565017261219795, MRR@10: 0.01709522896231757, Coverage@10: 0.23920070526006465, NCDG@10: 0.027168668032282132, HR_pos@10: 0.06098964326812428, HR_neg@10: 0.0005753739930955121, MRR_pos@10: 0.016903437631285736, MRR_neg@10: 0.00019179133103183735, NCDG_pos@10: 0.026880981035734382, NDCG_neg@10: 0.00028768699654775604, Matthews@10: 0.054900945858515716\n',
 'HR@20:0.10644418872266974, MRR@20: 0.02013918382197373, Coverage@20: 0.34205113135468707, NCDG@20: 0.03842549945746825, HR_pos@20: 0.10529344073647871, HR_neg@20: 0.0011507479861910242, MRR_pos@20: 0.019911431616373425, MRR_neg@20: 0.00022775220560030686, NCDG_pos@20: 0.03799704690158895, NDCG_neg@20: 0.0004284525558793015, Matthews@20: 0.07185783431178848\n']
lines3 = ['HR@5:0.0477560414269275, MRR@5: 0.02287111622554661, Coverage@5: 0.2618277990008816, NCDG@5: 0.029010753064382923, HR_pos@5: 0.04200230149597238, HR_neg@5: 0.005753739930955121, MRR_pos@5: 0.01975450709627925, MRR_neg@5: 0.0031166091292673566, NCDG_pos@5: 0.025239815675796563, NDCG_neg@5: 0.003770937388586361, Matthews@5: 0.03444772518107551\n',
 'HR@10:0.07997698504027617, MRR@10: 0.0270076442544797, Coverage@10: 0.3811342932706436, NCDG@10: 0.039266367882097405, HR_pos@10: 0.07192174913693901, HR_neg@10: 0.00805523590333717, MRR_pos@10: 0.023619330739583902, MRR_neg@10: 0.003388313514895793, NCDG_pos@10: 0.03478600026054245, NDCG_neg@10: 0.004480367621554957, Matthews@10: 0.06053506378511221\n',
 'HR@20:0.13176064441887225, MRR@20: 0.03036277182343352, Coverage@20: 0.5362915074933882, NCDG@20: 0.052061540646479706, HR_pos@20: 0.11334867663981588, HR_neg@20: 0.018411967779056387, MRR_pos@20: 0.026295278868273878, MRR_neg@20: 0.004067492955159644, NCDG_pos@20: 0.04501220064245336, NDCG_neg@20: 0.0070493400040263395, Matthews@20: 0.04037721499969662\n']
lines4 = ['HR@5:0.02589182968929804, MRR@5: 0.012332182585347141, Coverage@5: 0.14516602997355274, NCDG@5: 0.015647228820667873, HR_pos@5: 0.01668584579976985, HR_neg@5: 0.009205983889528193, MRR_pos@5: 0.007671653241273496, MRR_neg@5: 0.004660529344073649, NCDG_pos@5: 0.009885321796750352, NDCG_neg@5: 0.005761907023917523, Matthews@5: 0.025008567018682162\n',
 'HR@10:0.061565017261219795, MRR@10: 0.01696417155277915, Coverage@10: 0.21569203643843668, NCDG@10: 0.027055400395345798, HR_pos@10: 0.043153049482163405, HR_neg@10: 0.018411967779056387, MRR_pos@10: 0.011187599320510711, MRR_neg@10: 0.005776572232268434, NCDG_pos@10: 0.018429025027251973, NDCG_neg@10: 0.008626375368093825, Matthews@10: 0.06850258554942971\n',
 'HR@20:0.0995397008055236, MRR@20: 0.01947012253810841, Coverage@20: 0.31413458712900383, NCDG@20: 0.036495444543476134, HR_pos@20: 0.06501726121979287, HR_neg@20: 0.03452243958573072, MRR_pos@20: 0.012646674475753662, MRR_neg@20: 0.006823448062354744, NCDG_pos@20: 0.023884082647780025, NDCG_neg@20: 0.012611361895696109, Matthews@20: 0.05686428967211058\n']
lines5 = ['HR@5:0.023590333716915997, MRR@5: 0.012600690448791715, Coverage@5: 0.11871877754922128, NCDG@5: 0.015319358773925523, HR_pos@5: 0.007479861910241657, HR_neg@5: 0.01611047180667434, MRR_pos@5: 0.004593402378212505, MRR_neg@5: 0.008007288070579209, NCDG_pos@5: 0.0053096094285548364, NDCG_neg@5: 0.010009749345370682, Matthews@5: 0.033064641873732065\n',
 'HR@10:0.04602991944764097, MRR@10: 0.015517288618554443, Coverage@10: 0.1839553335292389, NCDG@10: 0.02249457272530249, HR_pos@10: 0.019562715765247412, HR_neg@10: 0.026467203682393557, MRR_pos@10: 0.006101704202970025, MRR_neg@10: 0.009415584415584414, NCDG_pos@10: 0.009109782289816015, NDCG_neg@10: 0.013384790435486476, Matthews@10: 0.10324751689514314\n',
 'HR@20:0.08342922899884925, MRR@20: 0.01800561518445674, Coverage@20: 0.27887158389656186, NCDG@20: 0.03180986267090908, HR_pos@20: 0.029344073647871116, HR_neg@20: 0.05408515535097814, MRR_pos@20: 0.006789341419396426, MRR_neg@20: 0.011216273765060313, NCDG_pos@20: 0.01159246460682124, NDCG_neg@20: 0.020217398064087833, Matthews@20: 0.08910238652782497\n']

In [125]:
alpha = 5
alphas = []
topks = []
metrics = []
for line in lines5:
    if 'HR@' in line:
        for i in line.split(', '):
            metrics.append(float(i.split(':')[-1].strip()))

In [126]:
for i in range(3):
    display(pd.DataFrame(metrics[11*i:11*(i+1)]).T)

,0,1,2,3,4,5,6,7,8,9,10
0,0.02359,0.012601,0.118719,0.015319,0.00748,0.01611,0.004593,0.008007,0.00531,0.01001,0.033065


,0,1,2,3,4,5,6,7,8,9,10
0,0.04603,0.015517,0.183955,0.022495,0.019563,0.026467,0.006102,0.009416,0.00911,0.013385,0.103248


,0,1,2,3,4,5,6,7,8,9,10
0,0.083429,0.018006,0.278872,0.03181,0.029344,0.054085,0.006789,0.011216,0.011592,0.020217,0.089102


In [81]:
11*3*4

132

## dict

In [127]:
d = {'r=16_2, alpha=2': 0.05531103432672559,
 'r=16_2, alpha=3': 0.07365974504539831,
 'r=16_2, alpha=4': 0.149377867671188,
 'r=16_2, alpha=5': 0.2383536087975349,
 'r=32_2, alpha=2': 0.05923706005849889,
 'r=32_2, alpha=3': 0.07472952705088284,
 'r=32_2, alpha=4': 0.15453795127479317,
 'r=32_2, alpha=5': 0.22130201655159662,
 'r=64_2, alpha=2': 0.05747916632248626,
 'r=64_2, alpha=3': 0.07821901652467918,
 'r=64_2, alpha=4': 0.1417710069101706,
 'r=64_2, alpha=5': 0.18265776635878014,
 'r=128_2, alpha=2': 0.06428255989173427,
 'r=128_2, alpha=3': 0.0921665249686498,
 'r=128_2, alpha=4': 0.14548104368411133,
 'r=128_2, alpha=5': 0.14519831508760941,
 'r=256_2, alpha=2': 0.04648171239236602,
 'r=256_2, alpha=3': 0.06436521303976957,
 'r=256_2, alpha=4': 0.11616547800080264,
 'r=256_2, alpha=5': 0.10279871155849937,
 'r=512_2, alpha=2': 0.01836295380723834,
 'r=512_2, alpha=3': -5.255336877465284e-06,
 'r=512_2, alpha=4': 0.024733820641945525,
 'r=512_2, alpha=5': 0.028398646574531276,
 'r=1024_2, alpha=2': -0.012605742462697647,
 'r=1024_2, alpha=3': 0.0011548263678972032,
 'r=1024_2, alpha=4': 0.008776905395472202,
 'r=1024_2, alpha=5': -0.004061604266722465,
 'r=2048_2, alpha=2': -0.008593398433128252,
 'r=2048_2, alpha=3': -0.001366927970622446,
 'r=2048_2, alpha=4': -0.027240563322450166,
 'r=2048_2, alpha=5': -0.028043964322806262,
 'r=16_3, alpha=2': 0.05230722171275475,
 'r=16_3, alpha=3': 0.059236661436579026,
 'r=16_3, alpha=4': 0.13705354078026863,
 'r=16_3, alpha=5': 0.20680013894842025,
 'r=32_3, alpha=2': 0.056041274291019426,
 'r=32_3, alpha=3': 0.07520242034916262,
 'r=32_3, alpha=4': 0.15791901975652467,
 'r=32_3, alpha=5': 0.2392319270741349,
 'r=64_3, alpha=2': 0.059928752161400034,
 'r=64_3, alpha=3': 0.07623041769189473,
 'r=64_3, alpha=4': 0.13637825903424544,
 'r=64_3, alpha=5': 0.19125081430382865,
 'r=128_3, alpha=2': 0.05747916632248626,
 'r=128_3, alpha=3': 0.07821901652467918,
 'r=128_3, alpha=4': 0.13086308422145937,
 'r=128_3, alpha=5': 0.16319532546876533,
 'r=256_3, alpha=2': 0.051921669262761865,
 'r=256_3, alpha=3': 0.0663952086002175,
 'r=256_3, alpha=4': 0.10554181919367596,
 'r=256_3, alpha=5': 0.13862609732445122,
 'r=512_3, alpha=2': 0.02158300430799532,
 'r=512_3, alpha=3': 0.04129713740698912,
 'r=512_3, alpha=4': 0.0692205693926399,
 'r=512_3, alpha=5': 0.08724229056479206,
 'r=1024_3, alpha=2': -0.0019113233866926468,
 'r=1024_3, alpha=3': 0.0016772129834947385,
 'r=1024_3, alpha=4': 0.03980498224388007,
 'r=1024_3, alpha=5': 0.06144435927282375,
 'r=2048_3, alpha=2': -0.019126859140523752,
 'r=2048_3, alpha=3': 0.00667494894545064,
 'r=2048_3, alpha=4': -0.028005508813704384,
 'r=2048_3, alpha=5': -0.03664498184133764,
 'r=16_4, alpha=2': 0.05307131610784122,
 'r=16_4, alpha=3': 0.060990852217879435,
 'r=16_4, alpha=4': 0.13434827072904545,
 'r=16_4, alpha=5': 0.214929221552216,
 'r=32_4, alpha=2': 0.05457285406582348,
 'r=32_4, alpha=3': 0.05671151567264219,
 'r=32_4, alpha=4': 0.13493890265515737,
 'r=32_4, alpha=5': 0.23749638986025642,
 'r=64_4, alpha=2': 0.05035481397541435,
 'r=64_4, alpha=3': 0.07125116535671248,
 'r=64_4, alpha=4': 0.12275109308233201,
 'r=64_4, alpha=5': 0.20031826195757757,
 'r=128_4, alpha=2': 0.056041274291019426,
 'r=128_4, alpha=3': 0.07520242034916262,
 'r=128_4, alpha=4': 0.11327806333028335,
 'r=128_4, alpha=5': 0.15958138310588654,
 'r=256_4, alpha=2': 0.04171101887483536,
 'r=256_4, alpha=3': 0.06271205106119178,
 'r=256_4, alpha=4': 0.08543329235115478,
 'r=256_4, alpha=5': 0.09895754479298603,
 'r=512_4, alpha=2': 0.01700777908163277,
 'r=512_4, alpha=3': 0.033540635418489305,
 'r=512_4, alpha=4': 0.0736567865476854,
 'r=512_4, alpha=5': 0.07278996180301688}

In [128]:
sorted(d.items(), key=lambda x: x[1], reverse=True)

[('r=32_3, alpha=5', 0.2392319270741349),
 ('r=16_2, alpha=5', 0.2383536087975349),
 ('r=32_4, alpha=5', 0.23749638986025642),
 ('r=32_2, alpha=5', 0.22130201655159662),
 ('r=16_4, alpha=5', 0.214929221552216),
 ('r=16_3, alpha=5', 0.20680013894842025),
 ('r=64_4, alpha=5', 0.20031826195757757),
 ('r=64_3, alpha=5', 0.19125081430382865),
 ('r=64_2, alpha=5', 0.18265776635878014),
 ('r=128_3, alpha=5', 0.16319532546876533),
 ('r=128_4, alpha=5', 0.15958138310588654),
 ('r=32_3, alpha=4', 0.15791901975652467),
 ('r=32_2, alpha=4', 0.15453795127479317),
 ('r=16_2, alpha=4', 0.149377867671188),
 ('r=128_2, alpha=4', 0.14548104368411133),
 ('r=128_2, alpha=5', 0.14519831508760941),
 ('r=64_2, alpha=4', 0.1417710069101706),
 ('r=256_3, alpha=5', 0.13862609732445122),
 ('r=16_3, alpha=4', 0.13705354078026863),
 ('r=64_3, alpha=4', 0.13637825903424544),
 ('r=32_4, alpha=4', 0.13493890265515737),
 ('r=16_4, alpha=4', 0.13434827072904545),
 ('r=128_3, alpha=4', 0.13086308422145937),
 ('r=64_4, a